# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secretofthedev import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

[{'avgTotalVolume': 63950500,
  'calculationPrice': 'close',
  'change': 0.76,
  'changePercent': 0.0045,
  'close': None,
  'closeSource': 'official',
  'closeTime': None,
  'companyName': 'Apple Inc',
  'currency': 'USD',
  'delayedPrice': None,
  'delayedPriceTime': None,
  'extendedChange': None,
  'extendedChangePercent': None,
  'extendedPrice': None,
  'extendedPriceTime': None,
  'high': None,
  'highSource': None,
  'highTime': None,
  'iexAskPrice': 0,
  'iexAskSize': 0,
  'iexBidPrice': 0,
  'iexBidSize': 0,
  'iexClose': 169.06,
  'iexCloseTime': 1712577936925,
  'iexLastUpdated': 1712577936925,
  'iexMarketPercent': 0.000923479556872435,
  'iexOpen': 169.05,
  'iexOpenTime': 1712577936925,
  'iexRealtimePrice': 169.05,
  'iexRealtimeSize': 35,
  'iexVolume': 259,
  'lastTradeTime': 1712347199767,
  'latestPrice': 169.58,
  'latestSource': 'Close',
  'latestTime': 'April 5, 2024',
  'latestUpdate': 1712347200333,
  'latestVolume': None,
  'low': None,
  'lowSource': 'Close'

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [6]:
pe_ratio = data[0]['peRatio']
pe_ratio

26.41

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [20]:
# # Function sourced from 
# # https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
# def chunks(lst, n):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), n):
#         yield lst[i:i + n]   
        
# symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_strings = []
# for i in range(0, len(symbol_groups)):
#     symbol_strings.append(','.join(symbol_groups[i]))
# #     print(symbol_strings[i])

# my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)


symbol_list = stocks['Ticker'].to_list()
symbol_string = ','.join(symbol_list)
batch_api_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
print(batch_api_url)
final_dataframe

https://api.iex.cloud/v1/data/core/QUOTE/A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [21]:
data = requests.get(batch_api_url).json()
for stock in data:
    final_dataframe = pd.concat([final_dataframe, pd.DataFrame([[stock['symbol'], stock['latestPrice'], stock['peRatio'], 'N/A']], columns = my_columns)], ignore_index = True)
    
final_dataframe

/var/folders/_5/hcq3myc97jn9bf7mbp59_jn00000gn/T/ipykernel_39020/2742799955.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, pd.DataFrame([[stock['symbol'], stock['latestPrice'], stock['peRatio'], 'N/A']], columns = my_columns)], ignore_index = True)


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,144.12,34.40,N/A
1,AAL,13.76,12.07,N/A
2,AAP,77.34,155.05,N/A
3,AAPL,169.58,26.41,N/A
4,ABBV,170.00,62.50,N/A
...,...,...,...,...
500,YUM,136.10,24.30,N/A
501,ZBH,127.33,26.09,N/A
502,ZBRA,290.38,50.77,N/A
503,ZION,42.08,9.65,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [22]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending=False, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MRK,127.99,893.16,N/A
1,AMD,170.42,324.30,N/A
2,KSU,293.59,279.61,N/A
3,ARE,124.58,230.32,N/A
4,CLX,145.94,227.14,N/A
5,FTI,26.93,216.65,N/A
6,AAP,77.34,155.05,N/A
7,WELL,91.76,139.94,N/A
8,LLY,784.21,135.21,N/A
9,IRM,78.64,125.48,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [25]:
def portfolio_input():
    global portfolio_size
    while True:
        try:
            portfolio_size = input('Enter the value of your portfolio:')
            val = float(portfolio_size)
            print(f'Portfolio value saved: {val}')
            break
        except ValueError:
            print('That is not a number. \nPlease try again.')

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [26]:
portfolio_input()

Portfolio value saved: 1000000.0


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [28]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MRK,127.99,893.16,156
1,AMD,170.42,324.30,117
2,KSU,293.59,279.61,68
3,ARE,124.58,230.32,160
4,CLX,145.94,227.14,137
5,FTI,26.93,216.65,742
6,AAP,77.34,155.05,258
7,WELL,91.76,139.94,217
8,LLY,784.21,135.21,25
9,IRM,78.64,125.48,254


## Building a Better (and More Realistic) Value Strategy
### Robust Value (RV) Startegy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [49]:
rv_columns = [
                'Ticker',
                'Company Name',
                'Company Name V', 
                'Price',
                'Number of Shares to Buy', 
                'Price-to-Earnings Ratio',
                'PE Percentile',
                'Price-to-Book Ratio',
                'PB Percentile',
                'Price-to-Sales Ratio',
                'PS Percentile',
                'EV/EBITDA',
                'EV/EBITDA Percentile',
                'EV/GP',
                'EV/GP Percentile',
                'RV Score'
                ]

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe

,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [50]:
symbol_list = stocks['Ticker'].to_list()
symbol_string = ','.join(symbol_list)
rv_batch_api_url = f'https://api.iex.cloud/v1/data/core/ADVANCED_STATS,QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'

rv_data = requests.get(rv_batch_api_url).json()
print(len(data))
print(len(symbol_list))
nr_of_endpoints = 2
if len(rv_data) == nr_of_endpoints * len(symbol_list):
    split = len(rv_data) // nr_of_endpoints
    rv_data_advanced_stats = rv_data[:split]
    rv_data_quote = rv_data[split:]
    for stock_stats, stock_quote in zip(rv_data_advanced_stats, rv_data_quote):
        enterprise_value = stock_stats['enterpriseValue']
        ebitda = stock_stats['EBITDA']
        gross_profit = stock_stats['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = pd.concat([rv_dataframe, pd.DataFrame([[
            stock_quote['symbol'],
            stock_quote['companyName'],
            stock_stats['companyName'],
            stock_quote['latestPrice'],
            'N/A',
            stock_stats['peRatio'],
            'N/A',
            stock_stats['priceToBook'],
            'N/A',
            stock_stats['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'
            ]], columns = rv_columns)], ignore_index = True)
      
# drop N/A values  
# hqm_dataframe.dropna(inplace=True)
rv_dataframe

505
505


/var/folders/_5/hcq3myc97jn9bf7mbp59_jn00000gn/T/ipykernel_39020/2036811444.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rv_dataframe = pd.concat([rv_dataframe, pd.DataFrame([[


,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,Agilent Technologies Inc.,Agilent Technologies Inc.,144.93,N/A,34.170815,N/A,6.83,N/A,6.2700,N/A,27.345697,N/A,12.640860,N/A,N/A
1,AAL,American Airlines Group Inc,American Airlines Group Inc,13.87,N/A,10.960406,N/A,-1.73,N/A,0.1707,N/A,5.779070,N/A,0.650403,N/A,N/A
2,AAP,Advance Auto Parts Inc,Advance Auto Parts Inc,78.39,N/A,155.042761,N/A,1.83,N/A,0.4084,N/A,14.003451,N/A,1.302771,N/A,N/A
3,AAPL,Apple Inc,Apple Inc,169.00,N/A,25.949424,N/A,35.34,N/A,6.7900,N/A,20.395009,N/A,15.279317,N/A,N/A
4,ABBV,Abbvie Inc,Abbvie Inc,171.40,N/A,62.450205,N/A,29.06,N/A,5.5400,N/A,15.760360,N/A,10.252160,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,Yum Brands Inc.,Yum Brands Inc.,135.88,N/A,23.976123,N/A,-4.87,N/A,5.4100,N/A,19.940093,N/A,9.236678,N/A,N/A
501,ZBH,Zimmer Biomet Holdings Inc,Zimmer Biomet Holdings Inc,126.87,N/A,25.550744,N/A,2.10,N/A,3.5400,N/A,15.656551,N/A,5.934300,N/A,N/A
502,ZBRA,Zebra Technologies Corp. - Class A,Zebra Technologies Corp.,295.12,N/A,50.421672,N/A,4.92,N/A,3.2600,N/A,24.684782,N/A,8.011218,N/A,N/A
503,ZION,Zions Bancorporation N.A,Zions Bancorporation N.A,42.27,N/A,9.644358,N/A,1.18,N/A,1.3400,N/A,6.053574,N/A,1.382673,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [51]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
5,ABC,AMERISOURCEBERGEN CORP,None,179.980,N/A,21.537842,N/A,53.04,N/A,0.1430,N/A,11.394933,N/A,4.604308,N/A,N/A
6,ABMD,Abiomed Inc.,Abiomed Inc.,381.020,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
28,ALXN,Alexion Pharmaceuticals Inc.,Alexion Pharmaceuticals Inc.,182.500,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,Aon plc. - Class A,Aon plc.,319.050,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
48,ATVI,Activision Blizzard Inc,Activision Blizzard Inc,94.420,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,Cerner Corp.,Cerner Corp.,94.920,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,"CenturyLink, Inc.","CenturyLink, Inc.",11.000,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
121,CTXS,"Citrix Systems, Inc.","Citrix Systems, Inc.",103.900,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,Warner Bros.Discovery Inc - Series C,Warner Bros.Discovery Inc,24.420,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
137,DISH,Dish Network Corp - Class A,Dish Network Corp,5.770,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [52]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

/var/folders/_5/hcq3myc97jn9bf7mbp59_jn00000gn/T/ipykernel_39020/2238724823.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)


In [53]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
5,ABC,AMERISOURCEBERGEN CORP,None,179.98,N/A,21.537842,N/A,53.04,N/A,0.143,N/A,11.394933,N/A,4.604308,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [54]:
from scipy.stats import percentileofscore as score

metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      76.435644
1      18.613861
2      98.613861
3      63.762376
4      93.267327
         ...    
500    54.851485
501    62.574257
502    88.316832
503    14.851485
504    72.673267
Name: PE Percentile, Length: 505, dtype: object
0      78.415842
1       6.336634
2      29.108911
3      97.425743
4      96.633663
         ...    
500     5.742574
501    32.178218
502    68.811881
503    13.267327
504    92.079208
Name: PB Percentile, Length: 505, dtype: object
0      83.663366
1       1.188119
2       4.950495
3      85.346535
4      79.306931
         ...    
500    78.415842
501    60.594059
502    57.425743
503    23.861386
504    91.485149
Name: PS Percentile, Length: 505, dtype: object
0      89.306931
1       5.940594
2      48.514851
3      78.217822
4       59.60396
         ...    
500    76.633663
501    58.415842
502    86.732673
503     7.128713
504    86.930693
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      83.564356
1       1.782178
2       7.524752
3 

,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,Agilent Technologies Inc.,Agilent Technologies Inc.,144.93,N/A,34.170815,76.435644,6.83,78.415842,6.2700,83.663366,27.345697,89.306931,12.640860,83.564356,N/A
1,AAL,American Airlines Group Inc,American Airlines Group Inc,13.87,N/A,10.960406,18.613861,-1.73,6.336634,0.1707,1.188119,5.779070,5.940594,0.650403,1.782178,N/A
2,AAP,Advance Auto Parts Inc,Advance Auto Parts Inc,78.39,N/A,155.042761,98.613861,1.83,29.108911,0.4084,4.950495,14.003451,48.514851,1.302771,7.524752,N/A
3,AAPL,Apple Inc,Apple Inc,169.00,N/A,25.949424,63.762376,35.34,97.425743,6.7900,85.346535,20.395009,78.217822,15.279317,90.693069,N/A
4,ABBV,Abbvie Inc,Abbvie Inc,171.40,N/A,62.450205,93.267327,29.06,96.633663,5.5400,79.306931,15.760360,59.60396,10.252160,73.267327,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,Yum Brands Inc.,Yum Brands Inc.,135.88,N/A,23.976123,54.851485,-4.87,5.742574,5.4100,78.415842,19.940093,76.633663,9.236678,68.514851,N/A
501,ZBH,Zimmer Biomet Holdings Inc,Zimmer Biomet Holdings Inc,126.87,N/A,25.550744,62.574257,2.10,32.178218,3.5400,60.594059,15.656551,58.415842,5.934300,44.158416,N/A
502,ZBRA,Zebra Technologies Corp. - Class A,Zebra Technologies Corp.,295.12,N/A,50.421672,88.316832,4.92,68.811881,3.2600,57.425743,24.684782,86.732673,8.011218,61.188119,N/A
503,ZION,Zions Bancorporation N.A,Zions Bancorporation N.A,42.27,N/A,9.644358,14.851485,1.18,13.267327,1.3400,23.861386,6.053574,7.128713,1.382673,8.712871,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [55]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.values():
        value_percentiles.append(rv_dataframe.loc[row, metric])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,Agilent Technologies Inc.,Agilent Technologies Inc.,144.93,N/A,34.170815,76.435644,6.83,78.415842,6.2700,83.663366,27.345697,89.306931,12.640860,83.564356,82.277228
1,AAL,American Airlines Group Inc,American Airlines Group Inc,13.87,N/A,10.960406,18.613861,-1.73,6.336634,0.1707,1.188119,5.779070,5.940594,0.650403,1.782178,6.772277
2,AAP,Advance Auto Parts Inc,Advance Auto Parts Inc,78.39,N/A,155.042761,98.613861,1.83,29.108911,0.4084,4.950495,14.003451,48.514851,1.302771,7.524752,37.742574
3,AAPL,Apple Inc,Apple Inc,169.00,N/A,25.949424,63.762376,35.34,97.425743,6.7900,85.346535,20.395009,78.217822,15.279317,90.693069,83.089109
4,ABBV,Abbvie Inc,Abbvie Inc,171.40,N/A,62.450205,93.267327,29.06,96.633663,5.5400,79.306931,15.760360,59.60396,10.252160,73.267327,80.415842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,Yum Brands Inc.,Yum Brands Inc.,135.88,N/A,23.976123,54.851485,-4.87,5.742574,5.4100,78.415842,19.940093,76.633663,9.236678,68.514851,56.831683
501,ZBH,Zimmer Biomet Holdings Inc,Zimmer Biomet Holdings Inc,126.87,N/A,25.550744,62.574257,2.10,32.178218,3.5400,60.594059,15.656551,58.415842,5.934300,44.158416,51.584158
502,ZBRA,Zebra Technologies Corp. - Class A,Zebra Technologies Corp.,295.12,N/A,50.421672,88.316832,4.92,68.811881,3.2600,57.425743,24.684782,86.732673,8.011218,61.188119,72.49505
503,ZION,Zions Bancorporation N.A,Zions Bancorporation N.A,42.27,N/A,9.644358,14.851485,1.18,13.267327,1.3400,23.861386,6.053574,7.128713,1.382673,8.712871,13.564356


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [56]:
rv_dataframe.sort_values('RV Score', ascending = False, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,LLY,Lilly(Eli) & Co,Lilly(Eli) & Co,778.380,N/A,142.189235,98.415842,69.17,98.811881,21.84,99.80198,64.746941,98.415842,28.379147,99.60396,99.009901
1,NVDA,NVIDIA Corp,NVIDIA Corp,874.680,N/A,73.931452,95.841584,51.19,97.821782,36.12,100.0,63.338892,98.217822,49.297420,100.0,98.376238
2,CDNS,"Cadence Design Systems, Inc.","Cadence Design Systems, Inc.",308.430,N/A,80.541267,96.831683,24.63,95.643564,20.50,99.60396,65.109030,98.613861,22.808989,98.217822,97.782178
3,DXCM,Dexcom Inc,Dexcom Inc,137.800,N/A,98.909801,97.623762,25.89,95.841584,14.79,98.019802,67.882329,99.009901,23.298684,98.415842,97.782178
4,NOW,ServiceNow Inc,ServiceNow Inc,781.075,N/A,92.961585,97.227723,21.10,94.257426,17.94,99.207921,118.978477,99.405941,22.344327,97.821782,97.584158
5,ISRG,Intuitive Surgical Inc,Intuitive Surgical Inc,384.015,N/A,76.565264,96.435644,10.34,88.514851,19.32,99.405941,60.514093,97.821782,28.003202,99.405941,96.316832
6,SNPS,"Synopsys, Inc.","Synopsys, Inc.",572.470,N/A,62.350529,93.069307,13.15,91.089109,14.31,97.623762,60.194811,97.623762,17.785490,94.257426,94.732673
7,IDXX,"Idexx Laboratories, Inc.","Idexx Laboratories, Inc.",516.130,N/A,51.062476,89.306931,29.07,96.831683,11.79,95.346535,36.008737,95.445545,19.928988,96.237624,94.633663
8,ANET,Arista Networks Inc,Arista Networks Inc,296.070,N/A,44.573768,85.544554,12.89,90.693069,15.88,98.217822,37.522370,95.841584,24.249306,99.009901,93.861386
9,VRSK,Verisk Analytics Inc,Verisk Analytics Inc,227.080,N/A,53.112117,90.49505,105.30,99.207921,12.17,96.237624,24.914519,87.128713,19.506182,95.643564,93.742574


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [57]:
portfolio_input()

Portfolio value saved: 1000000.0


In [58]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,LLY,Lilly(Eli) & Co,Lilly(Eli) & Co,778.380,25,142.189235,98.415842,69.17,98.811881,21.84,99.80198,64.746941,98.415842,28.379147,99.60396,99.009901
1,NVDA,NVIDIA Corp,NVIDIA Corp,874.680,22,73.931452,95.841584,51.19,97.821782,36.12,100.0,63.338892,98.217822,49.297420,100.0,98.376238
2,CDNS,"Cadence Design Systems, Inc.","Cadence Design Systems, Inc.",308.430,64,80.541267,96.831683,24.63,95.643564,20.50,99.60396,65.109030,98.613861,22.808989,98.217822,97.782178
3,DXCM,Dexcom Inc,Dexcom Inc,137.800,145,98.909801,97.623762,25.89,95.841584,14.79,98.019802,67.882329,99.009901,23.298684,98.415842,97.782178
4,NOW,ServiceNow Inc,ServiceNow Inc,781.075,25,92.961585,97.227723,21.10,94.257426,17.94,99.207921,118.978477,99.405941,22.344327,97.821782,97.584158
5,ISRG,Intuitive Surgical Inc,Intuitive Surgical Inc,384.015,52,76.565264,96.435644,10.34,88.514851,19.32,99.405941,60.514093,97.821782,28.003202,99.405941,96.316832
6,SNPS,"Synopsys, Inc.","Synopsys, Inc.",572.470,34,62.350529,93.069307,13.15,91.089109,14.31,97.623762,60.194811,97.623762,17.785490,94.257426,94.732673
7,IDXX,"Idexx Laboratories, Inc.","Idexx Laboratories, Inc.",516.130,38,51.062476,89.306931,29.07,96.831683,11.79,95.346535,36.008737,95.445545,19.928988,96.237624,94.633663
8,ANET,Arista Networks Inc,Arista Networks Inc,296.070,67,44.573768,85.544554,12.89,90.693069,15.88,98.217822,37.522370,95.841584,24.249306,99.009901,93.861386
9,VRSK,Verisk Analytics Inc,Verisk Analytics Inc,227.080,88,53.112117,90.49505,105.30,99.207921,12.17,96.237624,24.914519,87.128713,19.506182,95.643564,93.742574


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [63]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [64]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [65]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Company Name', string_template],
                    'C': ['Company Name V', string_template],
                    'D': ['Price', dollar_template],
                    'E': ['Number of Shares to Buy', integer_template],
                    'F': ['Price-to-Earnings Ratio', float_template],
                    'G': ['PE Percentile', percent_template],
                    'H': ['Price-to-Book Ratio', float_template],
                    'I': ['PB Percentile',percent_template],
                    'J': ['Price-to-Sales Ratio', float_template],
                    'K': ['PS Percentile', percent_template],
                    'L': ['EV/EBITDA', float_template],
                    'M': ['EV/EBITDA Percentile', percent_template],
                    'N': ['EV/GP', float_template],
                    'O': ['EV/GP Percentile', percent_template],
                    'P': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [66]:
writer.close()